For this task I will use the MNIST (Modified National Institute of Standards and Technology) dataset, which consists of 10 classes of greyscale handwritten digits, ranging from 0 to 9. 
The dataset contains 70,000 images in total - 60,000 images for the training set and 10,000 images for the testing set. Each image is a 28x28 pixel greyscale image (784 pixels per image), labelled from 0-9 indicating the digit represented. The pixel values range from 0-255 (black to white).

Below I will load the dataset and display some of its characteristics mentioned above:

In [1]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(f"Training set size: {x_train.shape[0]} images")
print(f"Test set size: {x_test.shape[0]} images")
print(f"Image dimensions: {x_train.shape[1]}x{x_train.shape[2]} pixels")
print(f"Labels: {set(y_train)}")

ModuleNotFoundError: No module named 'tensorflow'